In [4]:
from openml import datasets, tasks, runs, flows, setups
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
flowsJson = flows.list_flows()
flowList = pd.DataFrame(flowsJson).transpose()
pipelines = flowList.loc[flowList.full_name.str.startswith('sklearn.pipeline')]

In [53]:
pipelines.shape

(286, 6)